In [8]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
import re
import collections

In [9]:
df_doc = pd.read_excel("https://github.com/tbwa-digital-strategy/Blue-Bird-Bio/blob/master/HCPStateLevelData.xlsx?raw=true")
df_med_cent = pd.read_excel("https://github.com/tbwa-digital-strategy/Blue-Bird-Bio/blob/master/medical%20centers.xlsx?raw=true")
df_pop = pd.read_csv("https://raw.githubusercontent.com/tbwa-digital-strategy/Blue-Bird-Bio/master/scprc-est2016-18%2Bpop-res.csv")

# print(list(df_med_cent['data'].values))
# use only one of the following lines, whichever you prefer
# for i in list(df_med_cent['data'].values):
#     print(re.findall("[(][\d]{3}[)][ ]?[\d]{3}-[\d]{4}", str(i)))
filtered = [i for i in list(df_med_cent['data'].values) if len(re.findall(".*?(\(?\d{3}\D{0,3}\d{3}\D{0,3}\d{4}).*?", str(i))) == 0]

revised_filtered_lst = []

def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

for i, row in enumerate(filtered):
    if i % 4 == 2:
        revised_filtered_lst.append(str(row)[-2:])
    else:
        revised_filtered_lst.append(row)


        
bonemarrow_nums_by_state = {}        
for bucket in chunks(revised_filtered_lst, 4):
#     print(bucket)
    if bucket[2] in bonemarrow_nums_by_state:
        bonemarrow_nums_by_state[bucket[2]] += 1
    else:
        bonemarrow_nums_by_state[bucket[2]] = 1
    



for i, row_doc in df_doc.iterrows():
    for j, row_pop in df_pop.iterrows():
        if row_pop["NAME"].lower() in row_doc["State"].lower():
            df_doc.at[i, "Population"] = row_pop["POPESTIMATE2016"]
            
df_doc["Density Endos (per capita)"] = df_doc["Endos"]/df_doc["Population"]
df_doc["Density Opthamologists (per capita)"] = df_doc["Opthamologists"]/df_doc["Population"]
df_doc["Density Thalassemia (per capita)"] = df_doc["Thalassemia Patients"]/df_doc["Population"]
df_doc["Density Hematology Oncologists (per Thalassemia capita)"] = df_doc["Hematology Oncologists"]/df_doc["Thalassemia Patients"]
df_doc["Population Density"] = df_doc["Population"]/df_doc["Square Mileage"]

df_doc["code"] = df_doc["State"].apply(lambda x: x.split(",")[1])

for code in df_doc["code"]:
    if code not in bonemarrow_nums_by_state:
        bonemarrow_nums_by_state[code] = 0

df_doc.replace([np.inf, -np.inf], 0)
od = collections.OrderedDict(sorted(bonemarrow_nums_by_state.items()))
df_doc['Bone Marrow Medical Centers by State'] = list(od.values())
df_doc['Bone Marrow Medical Centers by Thalassemia Patients'] = df_doc['Bone Marrow Medical Centers by State']/df_doc["Thalassemia Patients"]
print(od)
print(df_doc["Density Hematology Oncologists (per Thalassemia capita)"])
print(df_doc.columns)

OrderedDict([('AK', 0), ('AL', 1), ('AR', 1), ('AZ', 4), ('CA', 17), ('CO', 3), ('CT', 1), ('DC', 2), ('DE', 2), ('FL', 10), ('GA', 4), ('HI', 1), ('IA', 1), ('ID', 0), ('IL', 8), ('IN', 2), ('KS', 2), ('KY', 2), ('LA', 4), ('MA', 6), ('MD', 3), ('ME', 0), ('MI', 5), ('MN', 4), ('MO', 4), ('MS', 1), ('MT', 1), ('NC', 5), ('ND', 0), ('NE', 1), ('NH', 1), ('NJ', 2), ('NM', 0), ('NV', 0), ('NY', 15), ('OH', 9), ('OK', 2), ('OR', 1), ('PA', 11), ('RI', 1), ('SC', 5), ('SD', 1), ('TN', 7), ('TX', 14), ('UT', 2), ('VA', 3), ('VT', 1), ('WA', 2), ('WI', 4), ('WV', 1), ('WY', 0)])
0       8.823529
1       5.666667
2       9.148148
3      86.000000
4       9.778443
5      41.800000
6       3.080000
7       7.400000
8     122.000000
9      17.076923
10      8.195122
11      6.571429
12     34.000000
13      8.700000
14     36.857143
15           inf
16     25.500000
17      4.184211
18     25.857143
19           inf
20     23.272727
21      9.880000
22     16.840000
23     19.687500
24     25.00

In [ ]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

scl=[[0.0, 'rgb(165,0,38)'], [0.1111111111111111, 'rgb(215,48,39)'], [0.2222222222222222, 'rgb(244,109,67)'], [0.3333333333333333, 'rgb(253,174,97)'], [0.4444444444444444, 'rgb(254,224,144)'], [0.5555555555555556, 'rgb(224,243,248)'], [0.6666666666666666, 'rgb(171,217,233)'], [0.7777777777777778, 'rgb(116,173,209)'], [0.8888888888888888, 'rgb(69,117,180)'], [1.0, 'rgb(49,54,149)']]

In [ ]:
data8 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_doc['code'],
        z = df_doc["Density Thalassemia (per capita)"],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Thalassemia Patients By State")
        ) ]

layout2 = dict(
        title = 'Thalassemia Patients per State',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig2 = dict( data=data8, layout=layout2 )
iplot( fig2, filename='thalassemia-cloropleth-map' )

In [ ]:
data7 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_doc['code'],
        z = df_doc["Density Hematology Oncologists (per Thalassemia capita)"],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Density Hematology Oncologists (per Thalassemia capita)")
        ) ]

layout2 = dict(
        title = 'Density Hematology Oncologists (per Thalassemia capita)',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig2 = dict( data=data7, layout=layout2 )
iplot( fig2, filename='hematology-cloropleth-map' )

In [ ]:
data9 = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_doc['code'],
        z = df_doc["Bone Marrow Medical Centers by Thalassemia Patients"],
        locationmode = 'USA-states',
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Bone Marrow Medical Centers by Thalassemia Patients")
        ) ]

layout2 = dict(
        title = 'Bone Marrow Medical Centers by Thalassemia Patients',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

fig2 = dict( data=data9, layout=layout2 )
iplot( fig2, filename='bone-marrow-medical-centers-cloropleth-map' )